In [74]:
import dimod, numpy
import networkx

done = False
choice = input("Choose problem to solve: \n(M) Max Cut\n(T) Traveling Sales Man\n(R) Room assignment\nEnter letter: ")
choice = choice.upper()

def SolveMAXCUT():
    NoNodes = 4
    h = {}
    j = {}
    y = 0
    for i in range(NoNodes-1):
        y = i+1
        while y<NoNodes:
            if (y+1)<=NoNodes:
                cost = numpy.random.uniform(-1, 1)
                j[(i,y)] = cost
                y +=1
    BQM = dimod.BinaryQuadraticModel(h,j,0.0, dimod.BINARY) 
    computer = dimod.ExactSolver()
    answer = computer.sample(BQM)
    LowestCost = answer.first.energy
    solution = answer.first.sample
    print(f"Solution:", solution)
    print(f"Its cost:", LowestCost)

def SolveTSP():
    NoCities = 4
    h = {} 
    j = {}   
    offset = 0.0

    # Initializing linear terms to 0 for city and position variables
    for i in range(NoCities):
        for p in range(NoCities):
            h[(i, p)] = 0.0

    # Random distances
    distances = {}
    for i in range(NoCities):
        for y in range(i+1, NoCities):
            d = numpy.random.uniform(1, 10)
            distances[(i, y)] = d
            distances[(y, i)] = d

    A = 10.0

    # First Constraint: each city i appears once
    for i in range(NoCities):
        for p in range(NoCities):
            h[(i, p)] += -A
        for w in range(NoCities):
            for k in range(w+1, NoCities):
                j[((i, w), (i, k))] = j.get(((i, w), (i, k)), 0.0) + 2.0 * A
        offset += A  # constant term (optional)

    # Second Constraint each position p has one city
    for p in range(NoCities):
        for i in range(NoCities):
            h[(i, p)] += -A
        for k in range(NoCities):
            for w in range(k+1, NoCities):
                j[((k, p), (w, p))] = j.get(((k, p), (w, p)), 0.0) + 2.0 * A
        offset += A

    # Add travel costs: for consecutive positions
    for p in range(NoCities - 1):
        for i in range(NoCities):
            for city in range(NoCities):
                if i == city:
                    continue
                cost = distances[(i, city)]
                y = ((i, p), (city, p+1))
                j[y] = j.get(y, 0.0) + cost

    BQM = dimod.BinaryQuadraticModel(h, j, offset, dimod.BINARY)
    solver = dimod.ExactSolver()
    result = solver.sample(BQM)

    best = result.first
    print("Energy (best):", best.energy)
    print("Binary solution (best):", best.sample)

    route = [None] * NoCities
    for (city, pos), val in best.sample.items():
        if val == 1:
            route[pos] = city
    print("Best route:", route)


def SolveRooms():
    N_rooms = 4
    N_classes = 4
    # set data
    chairs = [30, 40, 20, 50]     
    students = [25, 35, 15, 45]   
    A = 20.0
    h = {}
    j = {}
    offset = 0.0

    # Create variable for each (class, room)
    for c in range(N_classes):
        for r in range(N_rooms):
            h[(c, r)] = 0.0

    # Minimizing wasted chairs
    for c in range(N_classes):
        for r in range(N_rooms):
            waste = (chairs[r] - students[c])**2
            h[(c, r)] += waste

    # Constraint 1: each class assigned to one room
    for c in range(N_classes):
        for r in range(N_rooms):
            h[(c, r)] += -A  # linear part
        for k in range(N_rooms):
            for r2 in range(k+1, N_rooms):
                j[((c, k), (c, r2))] = j.get(((c, k), (c, r2)), 0.0) + 2*A
        offset += A

    # Constraint 2: each room has one class
    for r in range(N_rooms):
        for c in range(N_classes):
            h[(c, r)] += -A
        for c1 in range(N_classes):
            for c2 in range(c1+1, N_classes):
                j[((c1, r), (c2, r))] = j.get(((c1, r), (c2, r)), 0.0) + 2*A
        offset += A

    BQM = dimod.BinaryQuadraticModel(h, j, offset, dimod.BINARY)
    solver = dimod.ExactSolver()
    result = solver.sample(BQM)
    best = result.first

    print("Lowest energy:", best.energy)
    print("Binary assignment:", best.sample)


    for c in range(N_classes):
        for r in range(N_rooms):
            if best.sample[(c, r)] == 1:
                print(f"Class {c+1} assigned to Room {r+1}")

while done==False:
    if choice == "M":
        done = True
        SolveMAXCUT()
        
    elif choice == "T":
        done = True
        SolveTSP()
    elif choice == "R":
        done = True
        SolveRooms()
    else:
        choice = input("Choose a valid option")


Choose problem to solve: 
(M) Max Cut
(T) Traveling Sales Man
(R) Room assignment
Enter letter:  t


Energy (best): 9.335171748339041
Binary solution (best): {(0, 0): np.int8(1), (0, 1): np.int8(0), (0, 2): np.int8(0), (0, 3): np.int8(0), (1, 0): np.int8(0), (1, 1): np.int8(0), (1, 2): np.int8(1), (1, 3): np.int8(0), (2, 0): np.int8(0), (2, 1): np.int8(0), (2, 2): np.int8(0), (2, 3): np.int8(1), (3, 0): np.int8(0), (3, 1): np.int8(1), (3, 2): np.int8(0), (3, 3): np.int8(0)}
Best route: [0, 3, 1, 2]


In [33]:
import numpy, dimod
ProposedSolution=[2,2,2,2,2,2]
for w in range(len(ProposedSolution)):
    ProposedSolution[w] = int(input("Enter the binary variable for the 6 nodes: "))
    while ProposedSolution[w] != 1 and ProposedSolution[w] != 0:
        print("Please enter a valid input")
        ProposedSolution[w] = int(input("Enter the binary variable for the 6 nodes (1/0): "))
h = {0: 1, 1: 2, 2: 4, 3: 1, 4: 3, 5: 2}
j = {}
def CreatingJ(h,j): #Creates j based on h, assigns random values to each pair
    y = 0
    for i in range(len(h)-1):
        y = i+1
        while y<len(h):
            if (y+1)<=len(h):
                cost = numpy.random.uniform(0, 5)
                j[(i,y)] = cost
                y +=1
    return j
def CostOfProposedSolution(h, j, ProposedSolution): #Simulates Qubo for the proposed solution
    FirstSum = 0
    SecondSum = 0
    count = 0
    for i in ProposedSolution:
        x = i
        if x==1:
            FirstSum += h[count]
        count +=1
    for y in range(len(h)):
        for k in range(len(h)):
            if ProposedSolution[y] == 1 and ProposedSolution[k] == 1 and y!=k:
                if (k,y) in j:
                    cost = j[(k,y)]
                    SecondSum += cost
                elif (y,k) in j:
                    SecondSum += j[(y,k)]
    SecondSum = SecondSum/2
    Cost = FirstSum + SecondSum
    return Cost

#I added this recently to make it easy for you to check whether the code works
def CheckAnswer(h,j,ProposedSolution, MyAnswer):
    QUBO = dimod.BinaryQuadraticModel(h,j,0,dimod.BINARY)
    ActualAnswer = QUBO.energy(ProposedSolution)
    print(f"Actual Answer: ", ActualAnswer)
    Error = ((ActualAnswer-MyAnswer)/ActualAnswer)*100
    print(f"Relative Error: ", Error, "%")

j = CreatingJ(h, j)
print(j)
cost = CostOfProposedSolution(h, j, ProposedSolution)
print(cost)
CheckAnswer(h,j,ProposedSolution, cost)

Enter the binary variable for the 6 nodes:  1
Enter the binary variable for the 6 nodes:  1
Enter the binary variable for the 6 nodes:  1
Enter the binary variable for the 6 nodes:  1
Enter the binary variable for the 6 nodes:  1
Enter the binary variable for the 6 nodes:  1


{(0, 1): 4.94452936712375, (0, 2): 2.0278850651295044, (0, 3): 2.2374835149931145, (0, 4): 0.5234145406382928, (0, 5): 1.8171939714670953, (1, 2): 3.3928432745338624, (1, 3): 1.9387353521279964, (1, 4): 1.586678474790233, (1, 5): 1.9409470702884146, (2, 3): 1.2363719041557681, (2, 4): 1.2779579143647872, (2, 5): 0.002432054084099966, (3, 4): 0.33276197579217626, (3, 5): 2.547706491854402, (4, 5): 0.24609052126130126}
39.0530314926048
Actual Answer:  39.0530314926048
Relative Error:  0.0 %


In [64]:
# A and B are input binaries and C is their output
import dimod, math
A = "A"
B = "B"
C = "C"

def ANDgate():
    # E = (C - A*B)^2 = Y + A*B - 2*A*B*Y
    h = {A: 0, B: 0, C: 1}
    j = {(A, B): 1, (A, C): -2, (B, C): -2}
    QUBO = dimod.BinaryQuadraticModel(h,j,0,dimod.BINARY)
    return QUBO
    

def ORgate():
    # E = (C - A*B)^2 = C + A*B - 2*A*B*C
    h = {A: 0, B: 0, C: 1}
    j = {(A, C): 1, (A, C): -2, (B, C): -2}
    QUBO = dimod.BinaryQuadraticModel(h,j,0,dimod.BINARY)
    return QUBO
    
def XORgate():
    # E = (C - (A + B - 2*A*B))^2 = C + A + B + 4*A*B - 2*A*C - 2*B*C + 4*A*B*C
    h = {A: 1, B: 1, C: 1}
    j = {(A, B): 4, (A, C): -2, (B, C): -2}
    QUBO = dimod.BinaryQuadraticModel(h,j,0,dimod.BINARY)
    return QUBO

#filter wrong results
#give the wanted answer
QUBO = XORgate()
computer = dimod.ExactSolver()
answer = computer.sample(QUBO)
print(answer)

   A  B  D energy num_oc.
5  1  1  1   -2.0       1
4  0  1  1   -1.0       1
6  1  0  1   -1.0       1
0  0  0  0    0.0       1
1  1  0  0    0.0       1
3  0  1  0    0.0       1
2  1  1  0    1.0       1
7  0  0  1    1.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


In [68]:
def QuboOffset(QUBO):
    LinearSum = 0
    QuadraticSum = 0

    for (i, j), coeff in QUBO.items():
        if i == j:
            LinearSum += 0.5 * coeff       
        else:
            QuadraticSum += 0.25 * coeff     

    MeanEnergy = LinearSum + QuadraticSum
    offset = -MeanEnergy
    return offset


QUBO = {
    ('x1', 'x1'): 2,
    ('x2', 'x2'): 3,
    ('x3', 'x3'): 1,
    ('x1', 'x2'): -4,
    ('x1', 'x3'): 2,
    ('x2', 'x3'): -3
}

offset = QuboOffset(QUBO)
print("Estimated offset:", offset)


Estimated offset: 99.25
